#### Reinforcement Learning and Decision Making &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Homework #1

# &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Planning in MDPs

## Description

You are given an $N$-sided die, along with a corresponding Boolean mask
vector, `is_bad_side` (i.e., a vector of ones and zeros). You can assume
that $1<N\leq30$, and the vector `is_bad_side` is also of size $N$ and
$1$ indexed (since there is no $0$ side on the die). The game of DieN is
played as follows:

1.  You start with $0$ dollars.

2.  At any time you have the option to roll the die or to quit the game.

    1.  **ROLL**:

        1.  If you roll a number not in `is_bad_side`, you receive that
            many dollars (e.g., if you roll the number $2$ and $2$ is
            not a bad side -- meaning the second element of the vector
            `is_bad_side` is $0$, then you receive $2$ dollars). Repeat
            step 2.

        2.  If you roll a number in `is_bad_side`, then you lose all the
            money obtained in previous rolls and the game ends.

    2.  **QUIT**:

        1.  You keep all the money gained from previous rolls and the
            game ends.

## Procedure

-   You will implement your solution using the `solve()` method
    in the code below.
    
-   Your return value should be the number of dollars you expect to
    win for a specific value of `is_bad_side`, if you follow an
    optimal policy. That is, what is the value of the optimal
    state-value function for the initial state of the game (starting
    with $0$ dollars)? Your answer must be correct to $3$ decimal
    places, truncated (e.g., 3.14159265 becomes 3.141).

-   To solve this problem, you will need to determine an optimal policy
    for the game of DieN, given a particular configuration of the die.
    As you will see, the action that is optimal will depend on your
    current bankroll (i.e., how much money you've won so far).

-   You can try solving this problem by creating an MDP of the game
    (states, actions, transition function, reward function, and assume a
    discount rate of $\gamma=1$) and then calculating the optimal
    state-value function.

## Resources

The concepts explored in this homework are covered by:

-   Lecture Lesson 1: Smoov & Curly's Bogus Journey

-   Chapter 3 (3.6 Optimal Policies and Optimal Value Functions) and
    Chapter 4 (4.3-4.4 Policy Iteration, Value Iteration) of
    http://incompleteideas.net/book/the-book-2nd.html

-   Chapters 1-2 of 'Algorithms for Sequential Decision Making', M.
    Littman, 1996

## Submission

-   The due date is indicated on the Canvas page for this assignment.
    Make sure you have your timezone in Canvas set to ensure the
    deadline is accurate.

-   Submit your finished notebook on Gradescope. Your grade is based on
    a set of hidden test cases. You will have unlimited submissions -
    only the last score is kept.

-   Use the template below to implement your code. We have also provided
    some test cases for you. If your code passes the given test cases,
    it will run (though possibly not pass all the tests) on Gradescope.

-   Gradescope is using *python 3.6.x* and *numpy==1.18.0*, and you can
    use any core library (i.e., anything in the Python standard library).
    No other library can be used.  Also, make sure the name of your
    notebook matches the name of the provided notebook.  Gradescope times
    out after 10 minutes.

In [123]:
#################
# DO NOT REMOVE
# Versions
# numpy==1.18.0
################
import numpy as np
from collections import defaultdict
class MDPAgent(object):
    #you can model the MDP by defining a max run of the games or better you can keep creating state on the go.
    #states will keep track of the player's balance so set the rewards properly 
    def __init__(self):
        self.actions = ["play","quit"]
        
    def R(self,s,s_next):
        #no need for action here, Transition function will account for it
        #action cost nothing by itself, reward depends only on states and next states 
        if s=="game_over" or s_next == "game_over" or s=="lost": 
            return 0
        elif s_next == "lost":
            return -s # we lost everything (state == balance)
        else: # we were lucky
            return s_next - s #rewards = whatever we will have - whatever we have now 
        
    def T(self, s,a,is_bad_side):
        '''
        given the current state and action, T will return the probability of arriving at a new state. 
        T creates new states as the game evloves. State is defined as the possible player's balance + loss and end states
        '''
        if a == "quit" or s == "lost" or s == "game_over":
            return {"game_over": 1.0} #deterministic 
        else: #action == "play"
            N = len(is_bad_side)
            p_loss = sum(is_bad_side)/N
            #calculate the probability of each state, 1/n for indiviual win
            # s+n+1 is the next possible balance from this state, +1 is because index in python starts from 0
            T = {s+n+1: 1.0/N for n in range(N) if is_bad_side[n]==0}
            #add lost state
            T["lost"]=p_loss
            
            return T
        
        

    def solve(self, is_bad_side,alpha=1.0):
        """Implement the agent"""
        epsilon=1e-5
        V = defaultdict(int) # has to be defualtdict to handle the missing states that will be created by T
        V[0] =0 #will start will balance of 0
        V["lost"] = 0
        V["game_over"] = 0
        n = 0
        while n <200:
            n +=1
            V_temp = {}
            delta = 0
            for s in list(V.keys()):
                V_temp[s] =max(sum((self.R(s,s_next)+alpha*V[s_next])*p for s_next, p in self.T(s,a,is_bad_side).items()) for a in self.actions)
                
                delta = max(delta, abs(V_temp[s] - V[s]))
            V.update(V_temp) # updates the dictionary with the elements from another dictionary object or from an iterable of key/value pairs.
            if delta < epsilon:
                print(n)
                return round(V[0],3)


In [124]:
mdp = MDPAgent()
#mdp.T("game_over","quit",[1, 1, 1, 0, 0, 0])
mdp.solve([1, 1, 1, 0, 0, 0])

4


2.583

## Test cases

We have provided some test cases for you to help verify your implementation.

In [125]:
## DO NOT MODIFY THIS CODE.  This code will ensure that your submission
## will work proberly with the autograder

import unittest

class TestDieNNotebook(unittest.TestCase):
    def test_case_1(self):
        agent = MDPAgent()
        np.testing.assert_almost_equal(
            agent.solve(is_bad_side=[1, 1, 1, 0, 0, 0]),
            2.583,
            decimal=3
        )
        
    def test_case_2(self):
        agent = MDPAgent()
        np.testing.assert_almost_equal(
            agent.solve(
                is_bad_side=[1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0]
            ),
            7.379,
            decimal=3
        )
        
    def test_case_3(self):
        agent = MDPAgent()

        np.testing.assert_almost_equal(
            agent.solve(
                is_bad_side=[1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0]
            ),
            6.314,
            decimal=3
        )

unittest.main(argv=[''], verbosity=2, exit=False)

test_case_1 (__main__.TestDieNNotebook) ... ok
test_case_2 (__main__.TestDieNNotebook) ... ok
test_case_3 (__main__.TestDieNNotebook) ... 

4
6
4


ok

----------------------------------------------------------------------
Ran 3 tests in 0.037s

OK
